In [ ]:
small = ['/Users/alextaim/data/ccb/27_0000000000.json']
large = ['01_0.json']

files = small

In [ ]:
from VwPipeline import Loggers, Handlers
from VwPipeline.VwCache import VwCache
from VwPipeline.Vw import Vw
from VwPipeline.VwOpts import dimension, product
import pandas as pd

#your vw path
vw_path = r'vw'

cache = VwCache(r'_cache')
vw = Vw(
    vw_path,
    cache,
    handlers=[Handlers.WidgetHandler()],
    )

opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf --dsjson --compressed --synthcover --power_t 0  -P 1 --preserve_performance_counters --save_resume']),
))
preds = vw.train(files, opts, ['-p'])
prediction_files = preds.iloc[0]['!Outputs']['-p']
prediction_files

In [ ]:
def ccb_predictions(files, index = 't'):
    for f in files:
        for l in open(f):


class CcbPredictions:
    def __init__(self, files, index='t'):
        self.files = files
        self.index = index
        
    def iter_slots(self):
        result = {}
        for f in files:
            for l in open(f):
                o = json.loads(l)
                for 
                    yield result

# Estimators

In [ ]:
class ips:
    slots = None
    def __init__(self, slots = None):
        self.slots = slots
        self.r = 0
        self.n = 0

    def add(self, r, p_log, p, n = 1):
        slots = self.slots if self.slots else range(len(r))
        for s in slots:
            self.r += n * r[s] * p[s] / p_log[s]
            self.n += n

    def get(self):
        return {'e': 0 if self.n == 0 else self.r / self.n}

class snips:
    slots = None
    def __init__(self, slots = None):
        self.slots = slots
        self.r = 0
        self.n = 0

    def add(self, r, p_log, p, n = 1):
        slots = self.slots if self.slots else range(len(r))
        for s in slots:
            self.r += n * r[s] * p[s] / p_log[s]
            self.n += n * p[s] / p_log[s]

    def get(self):
        return {'e': 0 if self.n == 0 else self.r / self.n}

In [ ]:
df.to_csv('test.csv', index=False)
pd.read_csv('test.csv', header=[0,1])

# Usage

## Generate slim dsjson

In [ ]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.Dsjson import ccb

processor = ccb.Processor()
pipeline = FilesPipeline()

slim = pipeline.process(small, processor.process, path_gen=lambda p: fr'slim/{p}', process=True)
slim

## Sample

In [ ]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.Dsjson import ccb, filters

sampler = filters.UniformSampler(0.5)

processor = ccb.Processor(filters=[lambda l: sampler.do(l)])
result = pipeline.process(small, processor.process, path_gen=lambda p: fr'sample/{p}', process=True)
result

## Predict

In [ ]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.Dsjson import ccb

predictor = ccb.Predictor(filters=[lambda l: True])
cfe = pipeline.process(slim, predictor.predict, path_gen=lambda p: fr'cfe/{p}', process=True)
cfe

In [ ]:
import itertools
import json
predictions = map(lambda l: json.loads(l), itertools.chain.from_iterable([open(f) for f in cfe]))

In [ ]:
agg_factories = {'random': {'ips': lambda: ips()}, 'baseline1_old': {'ips': lambda: ips()}}
pd.DataFrame(aggregate(predictions, agg_factories, 100))

In [ ]:
sampler = UniformSampler(0.5)

parser = DsJsonCcb(filters=[lambda l: sampler.do(l)])
result = pipeline.process(small, parser.process, process=True)
result

In [ ]:
pd.DataFrame(filter(lambda s: s['SkipLearn']==False,
    csvs_2_rows(slot_files, processors = [
        timestamp
    ])))

In [ ]:
files = small

parser = DsJsonCcb()
slot_files= files_2_csvs(files, parser.lines_2_slots, path_gen=lambda p: fr'processed\{p}.txt')

In [ ]:
slots = filter(lambda s: s['SkipLearn']==False, csvs_2_rows(slot_files))
preds = VwPredictionsCcb.files_2_slots(prediction_files)

ds = map(lambda kv: ChainMap(kv[0], kv[1]), zip(slots, preds))

In [ ]:
pd.DataFrame(ds)

In [ ]:
pd.DataFrame(VwPredictionsCcb.files_2_slots(prediction_files))

In [ ]:
pd.DataFrame(map(lambda kv: ChainMap(kv[0], kv[1]), zip(slots, preds)))

In [ ]:
parser = DsJsonCcb()
list(parser.lines_2_slots(open(small[0])))

In [ ]:
large_result = parser.lines_2_slots(open(large[0]))

In [ ]:
next(large_result)

In [ ]:
list(zip(DsJsonCcb.files_2_slots(inputs,context_processors = [cp]), VwPredictionsCcb.lines_2_slots(open(prediction_file))))

In [ ]:
list(DsJsonCcb.files_2_slots(inputs,context_processors = [cp]))

In [ ]:
import pandas as pd
import json
import uuid
import itertools
import pytz

class DsJson:
    @staticmethod
    def is_ccb_event(line):
        try:
            o = json.loads(line)
        except:
            return False
        return line.startswith('{"Timestamp"')

    @staticmethod
    def is_cb_event(line):
        return line.startswith('{"_label_cost"')

    @staticmethod
    def is_dangling_reward(line):
        return line.startswith('{"RewardValue')

    @staticmethod
    def get_timestamp(line):
        obj = NaiveJson(line)
        if line.startswith('{"RewardValue'):
            return pd.to_datetime(obj.get_string("EnqueuedTimeUtc"))
        return pd.to_datetime(obj.get_string("Timestamp"))

    @staticmethod
    def context(line):
        parsed = json.loads(line)
        return json.dumps(parsed['c']) + '\n'

    @staticmethod
    def dangling_reward(line):
        parsed = json.loads(line)
        return {'Timestamp': pd.to_datetime(parsed['EnqueuedTimeUtc']), 'EventId': parsed['EventId'], 'Reward': parsed['RewardValue']}

    @staticmethod
    def analyze_observations(obj):
        rewards = 0
        activations =0
        for o in obj:
            if 'ActionTaken' in o and o['ActionTaken']==True:
                activations = activations + 1
            else:
                rewards = rewards + 1

        return rewards, activations

    def get_title_from_obj(action):
        c = action['c']
        if 'Title' in c:
            return c['Title']
        elif 'ProductTitle' in c:
            return c['ProductTitle']
        return None

    @staticmethod
    def ccb_event(line):
        parsed = json.loads(line)
        session = {'Session': str(uuid.uuid4()),
                 'Timestamp': pd.to_datetime(parsed['Timestamp']),
                 'NumActions': len(parsed['c']['_multi']),
                 'NumSlots': len(parsed['c']['_slots']),
                 'VWState': parsed['VWState']['m'],
                 'SkipLearn': False if '_skipLearn' not in parsed else parsed['_skipLearn'],
                 'StringLen': len(line),
                 'Pdrop': 0.0 if 'pdrop' not in parsed else parsed['pdrop']}

        multi = [None] * len(parsed['c']['_multi'])
        for i, o in enumerate(parsed['c']['_multi']):
            multi[i] = {'Id': o['c']['Id'],
                        'Len': len(json.dumps(o))}

        slots = [None] * len(parsed['_outcomes'])
        for i, o in enumerate(parsed['_outcomes']):
            r, a = DsJson.analyze_observations(o['_o'])
            slots[i] = {'SlotIdx': i,
                    'Cost': o['_label_cost'],
                    'EventId': o['_id'],
                    'ActionsPerSlot': len(o['_a']),
                    'Chosen': o['_a'][0],
                    'Prob': o['_p'][0],
                    'Rewards': r,
                    'Activations': a,
                    'Product': multi[o['_a'][0]]['Id'],
                    'ChosenActionLen': multi[o['_a'][0]]['Len']}
        
        return [dict(session, **m) for m in multi] 

    @staticmethod
    def ccb_2_cb(session, slots, multi):
        return [dict(session, **s) for s in slots]

    @staticmethod
    def ccb_as_cb_to_stats(df):
        result = df
        result['TimestampFloor'] = result.index.floor('1min')
        result['TimestampFloor'] = result['TimestampFloor'].dt.tz_localize(None)
        result['Observations'] = result['HasObservation'].astype(int).div(1 - result['Pdrop'])
        result['Rewards'] = -result['Cost'].div(1 - result['Pdrop'])
        result['Events'] = 1
        result['EventsLogged'] = result['Events']
        result['Events'] = result['Events'].div(1 - result['Pdrop'])
        result['RewardsSlot1'] = result['Rewards'].mul((result['SlotIdx']==0).astype(int))
        result['EventsSlot1'] = result['Events'].mul((result['SlotIdx']==0).astype(int))
        result['RewardsIps1'] = result['Rewards'].mul((result['SlotIdx']==result['Chosen']).astype(int)).div(result['Prob'])
        result['EventsIps1'] = result['Events'].mul((result['SlotIdx']==result['Chosen']).astype(int)).div(result['Prob'])
        result['RewardsIps1Slot1'] = result['RewardsIps1'].mul((result['SlotIdx']==0).astype(int))
        result['EventsIps1Slot1'] = result['EventsIps1'].mul((result['SlotIdx']==0).astype(int))
        result['RewardsIpsR'] = result['Rewards'].mul(result['ActionsPerSlot']).div(result['Prob'])
        result['EventsIpsR'] = result['Events'].mul(result['ActionsPerSlot']).div(result['Prob'])
        result['RewardsIpsRSlot1'] = result['RewardsIpsR'].mul((result['SlotIdx']==0).astype(int))
        result['EventsIpsRSlot1'] = result['EventsIpsR'].mul((result['SlotIdx']==0).astype(int))

        return result[['TimestampFloor', 'Observations', 'Rewards', 'Events', 'RewardsSlot1', 'EventsSlot1', 'RewardsIps1', 'EventsIps1', 'RewardsIps1Slot1', 'EventsIps1Slot1', 'RewardsIpsR', 'EventsIpsR', 'RewardsIpsRSlot1', 'EventsIpsRSlot1', 'EventsLogged']].reset_index().drop('Timestamp', axis=1).rename(columns = {'TimestampFloor': 'Timestamp'}).groupby('Timestamp').sum()

    @staticmethod
    def get_title_from_obj(action):
        c = action['c']
        if 'Title' in c:
            return c['Title']
        elif 'ProductTitle' in c:
            return c['ProductTitle']
        return None
    
    @staticmethod
    def ccb_action(line):
        parsed = json.loads(line)
        session = {'Session': parsed['_outcomes'][0]['_id'], 'Timestamp': pd.to_datetime(parsed['Timestamp'])}
        multi = [None] * len(parsed['c']['_multi'])
        for i, o in enumerate(parsed['c']['_multi']):
            multi[i] = {'Id': o['c']['Id'],
                        'Index': i,
                        'ChannelId': o['c']['Id'],
                        'Title': DsJson.get_title_from_obj(o),
                        'SlotIdx': -1,
                        'Cost': 0,
                        'Prob': 0,
                        'ActionLen': len(str(o)),
                        'CLen': len(o['c']),
                        'DLen': len(o['d']),
                        'ELen': len(o['e']),
                        'HLen': len(o['h']),                        
       #                 'plc0': o['c']['plc0'],
       #                 'plc1': o['c']['plc1'],
       #                 'plc2': o['c']['plc2'],
       #                 'plc3': o['c']['plc3'],
       #                 'plc4': o['c']['plc4'],
                       }
      #      for key in o['c']:
      #          multi[i][f'c/{key}'] = o['c'][key]
        for i, o in enumerate(parsed['_outcomes']):
            multi[o['_a'][0]]['SlotIdx'] = i
            multi[o['_a'][0]]['Cost'] = o['_label_cost']
            multi[o['_a'][0]]['Prob'] = o['_p'][0]
        return [dict(session, **m) for m in multi]      

    @staticmethod
    def dangling_reward_lines(lines):
        return filter(lambda l: DsJson.is_dangling_reward(l), lines)

    @staticmethod
    def ccb_decision_lines(lines):
        return filter(lambda l: DsJson.is_ccb_event(l), lines)
    
    @staticmethod
    def dangling_rewards(lines):
        df = pd.DataFrame(
            map(lambda l: DsJson.dangling_reward(l), DsJson.dangling_reward_lines(lines)))
        return df.set_index('Timestamp') if len(df) > 0 else df

    @staticmethod
    def ccb_events(lines):
        events = map(lambda l: DsJson.ccb_2_cb(*DsJson.ccb_event(l)), DsJson.ccb_decision_lines(lines))
        df = pd.DataFrame(itertools.chain(*events))
        return df#.set_index('Timestamp')

    @staticmethod
    def ccb_stats(lines):
        events = map(lambda l: DsJson.ccb_2_cb(*DsJson.ccb_event(l)), DsJson.ccb_decision_lines(lines))
        df = pd.DataFrame(itertools.chain(*events))
        return DsJson.ccb_as_cb_to_stats(df.set_index('Timestamp'))

    @staticmethod
    def ccb_actions(lines):
        actions = map(lambda l: DsJson.ccb_action(l), DsJson.ccb_decision_lines(lines))
        df = pd.DataFrame(itertools.chain(*actions))
        return df.set_index('Timestamp')

    @staticmethod
    def contexts(lines):
        return map(lambda e: DsJson.context(e),
            filter(lambda l: DsJson.is_ccb_event(l), lines))
    
    @staticmethod
    def first_timestamp(lines):
        line = next(lines)
        return DsJson.get_timestamp(line)
    
def ccb_actions(file):
    return DsJson.ccb_actions(open(file, 'r', encoding='utf-8'))

def ccb_slots(file):
    return DsJson.ccb_events(open(file, 'r', encoding='utf-8'))

def dangling_rewards(file):
    return DsJson.dangling_rewards(open(file, 'r', encoding='utf-8'))

In [ ]:
import pandas as pd

df = pd.DataFrame([{'i': i} for i in range(16)])

In [ ]:
df

In [ ]:
df.sample(frac = 0.5)